# LM741 – Plan de mesures (mode examen)
## Montage non-inverseur – Av = 2 (R1 = 1 kΩ, R2 = 1 kΩ) – Sans condensateurs

## 0) Rappels essentiels
- Amplificateur opérationnel LM741
- Montage : non-inverseur
- Gain théorique :
  Av = 1 + R2 / R1
- Ici :
  R1 = 1 kΩ
  R2 = 1 kΩ
  → Av = 2

## 1) Câblage (non-inverseur, Av = 2)

LM741 (DIP-8) :
- Pin 7 : +Vs (ex. +15 V)
- Pin 4 : −Vs (ex. −15 V)
- Pin 3 (+) : Vin (sortie du générateur)
- Pin 2 (−) : nœud de contre-réaction
- Pin 6 : Vout (vers oscilloscope)

Réseau de gain :
- R1 = 1 kΩ : pin 2 (−) → GND
- R2 = 1 kΩ : pin 6 (Vout) → pin 2 (−)

Mesures :
- Oscilloscope CH1 → Vin
- Oscilloscope CH2 → Vout

Remarque importante :
- Aucun condensateur de découplage utilisé.
- Si bruit ou oscillations apparaissent : réduire la longueur des fils et vérifier les masses.
- Masse commune obligatoire : alim = générateur = oscilloscope.

## 2) Test 1 — Vérification du gain

Objectif :
- Vérifier que Av ≈ 2 à basse fréquence.

Réglages générateur :
- Signal : sinus
- Fréquence : 1 kHz (ou 10 kHz)
- Amplitude : 0.5 Vpp à 1 Vpp

Mesure :
- Av_mesuré = Vpp_out / Vpp_in

Résultat attendu :
- Av ≈ 2 (écart possible dû aux tolérances et limites du LM741)

## 3) Test 2 — Saturation et swing de sortie

Objectif :
- Observer la saturation de la sortie.

Méthode :
- Fixer f = 1 kHz
- Augmenter progressivement l’amplitude d’entrée
- Observer Vout

Observation :
- Quand Vout devient écrêté (sinus aplati), le LM741 est en saturation.

Conclusion :
- La sortie ne peut pas atteindre exactement ±Vs.
- Le swing de sortie est inférieur aux rails d’alimentation.

## 4) Test 3 — Bande passante (fréquence de coupure −3 dB)

Objectif :
- Déterminer la fréquence de coupure fc du montage.

Méthode :
- Garder Vin faible (ex. 200 mVpp) pour éviter saturation et slew rate
- Balayer la fréquence (ex. 100 Hz → 500 kHz)
- Mesurer Av(f)

Définition :
- fc = fréquence pour laquelle Av = Av0 / √2 (−3 dB)

Lien théorique :
- Le LM741 a un produit gain-bande passante quasi constant.
- Avec Av = 2, fc est plus élevée qu’avec un gain plus grand.

## 5) Test 4 — Slew Rate

Objectif :
- Mettre en évidence la limitation de slew rate.

Méthode (sinus) :
- Choisir une fréquence élevée (ex. 20 kHz à 50 kHz)
- Augmenter Vin progressivement
- Observer la sortie

Observation :
- Le sinus devient triangulaire → limitation par le slew rate.

Calcul :
- SR ≈ 2π · f · Vpeak_out

## 6) Test 5 — Offset de sortie (optionnel)

Objectif :
- Observer la tension de sortie pour Vin = 0.

Méthode :
- Mettre le générateur à 0 V
- Mesurer Vout (en mV)

Interprétation :
- Tension due à l’offset d’entrée et aux imperfections internes.

## 7) Bonnes pratiques (examen)
- Toujours commencer avec Vin faible.
- Séparer les phénomènes :
  - Saturation → basse fréquence, grande amplitude
  - Bande passante → petite amplitude
  - Slew rate → haute fréquence
- Expliquer chaque limite par une cause physique interne du LM741.


# Test 1


In [24]:
import pyvisa

# =========================
# PARAMÈTRES (optionnels)
# =========================
OPEN_TIMEOUT_MS = 3000

def try_open(rm, addr, timeout=OPEN_TIMEOUT_MS):
    """Essaye d’ouvrir une ressource sans faire planter le script."""
    try:
        inst = rm.open_resource(addr)
        inst.timeout = timeout
        inst.read_termination = "\n"
        inst.write_termination = "\n"
        try:
            inst.clear()
        except Exception:
            pass
        return inst
    except Exception as e:
        print(f"❌ Impossible d’ouvrir {addr} -> {type(e).__name__}: {e}")
        return None

def detect_simple():
    rm = pyvisa.ResourceManager()
    resources = rm.list_resources()
    print("Ressources trouvées :", resources)

    # Candidates (on ne fait PAS de *IDN?*)
    gen_candidates = [a for a in resources if ("DG" in a or "DG1" in a or "DG4" in a)]
    scope_candidates = [a for a in resources if ("DS" in a or "DS1" in a)]

    print("\nCandidats générateur :", gen_candidates)
    print("Candidats oscillo     :", scope_candidates)

    generator = None
    scope = None
    generator_addr = None
    scope_addr = None

    # Ouvrir générateur (1er qui marche)
    for addr in gen_candidates:
        generator = try_open(rm, addr)
        if generator is not None:
            generator_addr = addr
            break

    # Ouvrir oscillo (1er qui marche)
    for addr in scope_candidates:
        scope = try_open(rm, addr)
        if scope is not None:
            scope_addr = addr
            break

    print("\n=== Résultat détection simple ===")
    print("Générateur :", generator_addr)
    print("Oscilloscope :", scope_addr)

    # IMPORTANT: on ne ferme PAS ici si tu veux les réutiliser juste après.
    # Si tu veux juste tester la détection, décommente ces lignes:
    # if generator: generator.close()
    # if scope: scope.close()
    # rm.close()

    return rm, generator, scope

if __name__ == "__main__":
    rm, generator, scope = detect_simple()


Ressources trouvées : ('USB0::0x0400::0x09C4::DG1F143800623::INSTR', 'USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR', 'ASRL7::INSTR', 'ASRL8::INSTR')

Candidats générateur : ['USB0::0x0400::0x09C4::DG1F143800623::INSTR']
Candidats oscillo     : ['USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR']

=== Résultat détection simple ===
Générateur : USB0::0x0400::0x09C4::DG1F143800623::INSTR
Oscilloscope : USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR


# PRise en main 

In [ ]:
import pyvisa
import time

# -----------------------------
# PARAMÈTRES (Partie 1 : Gain)
# -----------------------------
GEN_ID   = "USB0::0x0400::0x09C4::DG1F143800623::INSTR"  # ton générateur
CH       = 1

FREQ_HZ  = 67e3        # 1 kHz
VPP_V    = 3          # 1 Vpp (à ajuster)
OFFSET_V = 0.0           # 0 VPP
FREQ_HZ2 = 200e3      # 200 kHz
VPP_V2   = 2          # 2 Vpp (à ajuster)
OFFSET_V2= 0.0         # 0 VPP


# -----------------------------
# Connexion au générateur
# -----------------------------
rm = pyvisa.ResourceManager()
gen = rm.open_resource(GEN_ID)

gen.timeout = 5000
gen.read_termination  = "\n"
gen.write_termination = "\n"

# Reset propre (comme Lab1)
gen.write("*RST")
gen.write("*CLS")
time.sleep(1)

# -----------------------------
# Prise en main : Sinus CH1
# -----------------------------
# Unité en Vpp (comme Lab1)
gen.write(f"SOUR{CH}:VOLT:UNIT VPP")

# SINUS CH2
#gen.write(f"SOUR2:FUNC SIN")

# Utiliser APPLy (simple + fiable)
# Format Rigol: APPLy:SINusoid <freq>,<ampl>,<offset>
gen.write(f"APPLy:SINusoid {FREQ_HZ},{VPP_V},{OFFSET_V}")
gen.write(f"APPLy2:SINusoid {FREQ_HZ2},{VPP_V2},{OFFSET_V2}")

# Activer sortie
gen.write(f"OUTP{CH} ON")
#gen.write("OUTP2 ON")
print(f"CH{CH} : SIN {FREQ_HZ} Hz, {VPP_V} Vpp, offset {OFFSET_V} V")
#print(f"CH2 : SIN {FREQ_HZ2} Hz, {VPP_V2} Vpp, offset {OFFSET_V2} V")




#gen.close()
#rm.close()


CH1 : SIN 67000.0 Hz, 3 Vpp, offset 0.0 V


In [29]:
import pyvisa
import time
import numpy as np
import matplotlib.pyplot as plt

GEN_ID   = "USB0::0x0400::0x09C4::DG1F143800623::INSTR"
SCOPE_ID = "USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR"

FREQ_HZ  = 1000
VPP_V    = 3.0
OFFSET_V = 0.0
GEN_CH   = 1

def read_scope_channel(scope, channel="CHAN1"):
    # Lecture waveform en ASCII (simple à parser)
    scope.write(f":WAV:SOUR {channel}")
    scope.write(":WAV:MODE NORM")
    scope.write(":WAV:FORM ASC")

    pre = scope.query(":WAV:PRE?").strip()
    raw = scope.query(":WAV:DATA?").strip()

    # Données CSV
    parts = [p for p in raw.split(",") if p.strip() != ""]
    v = np.array([float(p) for p in parts], dtype=float)

    # Preamble -> xincr/xorig (heuristique DS1000Z)
    nums = []
    for p in pre.split(","):
        try:
            nums.append(float(p))
        except:
            pass

    xincr = nums[4] if len(nums) > 4 else 1.0
    xorig = nums[5] if len(nums) > 5 else 0.0
    t = xorig + np.arange(len(v)) * xincr
    return t, v

def main():
    rm = pyvisa.ResourceManager()

    # ---- 1) Générateur : envoyer sinus CH1
    gen = rm.open_resource(GEN_ID)
    gen.timeout = 5000
    gen.read_termination  = "\n"
    gen.write_termination = "\n"

    gen.write("*RST")
    gen.write("*CLS")
    time.sleep(1)

    gen.write(f"SOUR{GEN_CH}:VOLT:UNIT VPP")
    gen.write(f"APPLy:SINusoid {FREQ_HZ},{VPP_V},{OFFSET_V}")
    gen.write(f"OUTP{GEN_CH} ON")
    gen.close()

    # laisser stabiliser le signal
    time.sleep(0.5)

    # ---- 2) Oscillo : lire CH1 et CH2
    scope = rm.open_resource(SCOPE_ID)
    scope.timeout = 15000
    scope.read_termination  = "\n"
    scope.write_termination = "\n"

    t1, v1 = read_scope_channel(scope, "CHAN1")
    t2, v2 = read_scope_channel(scope, "CHAN2")

    scope.close()
    rm.close()

    # ---- 3) Plot
    plt.figure(figsize=(10, 5))
    plt.plot(t1 * 1e3, v1, label="CH1 (entrée LM741)")
    plt.plot(t2 * 1e3, v2, label="CH2 (sortie LM741)")
    plt.xlabel("Temps (ms)")
    plt.ylabel("Tension (V)")
    plt.title(f"LM741 – SIN {FREQ_HZ} Hz, {VPP_V} Vpp, offset {OFFSET_V} V")
    plt.grid(True)
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [3]:
import pyvisa
rm = pyvisa.ResourceManager()
print("VISA lib:", rm.visalib)
print("Resources:", rm.list_resources())
rm.close()


VISA lib: Visa Library at C:\WINDOWS\system32\visa32.dll
Resources: ('USB0::0x0400::0x09C4::DG1F143800623::INSTR', 'USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR', 'ASRL7::INSTR', 'ASRL8::INSTR')


In [23]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt

SCOPE_ID = "USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR"

rm = pyvisa.ResourceManager()
scope = rm.open_resource(SCOPE_ID)

scope.timeout = 10000
scope.write_termination = "\n"
scope.read_termination  = "\n"
scope.encoding = "ascii"

print("SCOPE:", scope.query("*IDN?").strip())

# Figer l'acquisition pour lire un signal stable
scope.write(":STOP")
delay = 0.5  # secondes
time.sleep(delay)
scope.write(":WAV:FORM ASC")
scope.write(":WAV:MODE NORM")

def read_waveform(chan: str):
    scope.write(f":WAV:SOUR {chan}")
    pre = scope.query(":WAV:PRE?").strip().split(",")

    # Rigol preamble: ... xinc,xorig,xref,yinc,yorig,yref
    xinc  = float(pre[4])
    xorig = float(pre[5])
    xref  = float(pre[6])
    yinc  = float(pre[7])
    yorig = float(pre[8])
    yref  = float(pre[9])

    raw = scope.query(":WAV:DATA?")

    # Enlever header binaire si jamais présent (#9xxxxxx...)
    if raw.startswith("#"):
        nd = int(raw[1])
        nb = int(raw[2:2+nd])
        raw = raw[2+nd:2+nd+nb]

    adc = np.array([float(v) for v in raw.split(",") if v.strip() != ""])
    i = np.arange(adc.size)
    t = (i - xref) * xinc + xorig
    v = (adc - yref) * yinc + yorig
    return t, v

t1, v1 = read_waveform("CHAN1")  # entrée
t2, v2 = read_waveform("CHAN2")  # sortie

# Relancer l'oscillo
scope.write(":RUN")
time.leep(0.1)
scope.close()
time.sleep(0.1)
rm.close()

# ===== Mesures de base =====
def vpp(x): return float(np.max(x) - np.min(x))
Vpp_in  = vpp(v1)
Vpp_out = vpp(v2)
gain = Vpp_out / Vpp_in if Vpp_in else np.nan

# ===== Slew rate sur CH2 : max(|dV/dt|) =====
dv = np.diff(v2)
dt = np.diff(t2)
sr_inst = np.abs(dv/dt)              # V/s
sr = np.max(sr_inst) / 1e6           # V/µs

print(f"Vpp_in  = {Vpp_in:.3f} V")
print(f"Vpp_out = {Vpp_out:.3f} V")
print(f"Gain    = {gain:.3f}")
print(f"Slew rate (max |dV/dt|) ≈ {sr:.3f} V/µs")

# ===== Plot =====
plt.figure()
plt.plot(t1*1e6, v1, label="CH1 (entrée)")
plt.plot(t2*1e6, v2, label="CH2 (sortie)")
plt.xlabel("Temps (µs)")
plt.ylabel("Tension (V)")
plt.title("LM741 - Entrée vs Sortie")
plt.grid(True)
plt.legend()
plt.show()


SCOPE: RIGOL TECHNOLOGIES,DS1104Z Plus,DS1ZC212301111,00.04.04.SP4


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [22]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt

SCOPE_ID = "USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR"

rm = pyvisa.ResourceManager()
scope = rm.open_resource(SCOPE_ID)
scope.timeout = 10000
scope.write_termination = "\n"
scope.read_termination  = "\n"
scope.encoding = "ascii"

print("SCOPE:", scope.query("*IDN?").strip())

scope.write(":STOP")
scope.write(":WAV:FORM ASC")
scope.write(":WAV:MODE NORM")

def read_waveform(chan: str):
    scope.write(f":WAV:SOUR {chan}")
    pre = scope.query(":WAV:PRE?").strip().split(",")

    xinc  = float(pre[4])
    xorig = float(pre[5])
    xref  = float(pre[6])
    yinc  = float(pre[7])
    yorig = float(pre[8])
    yref  = float(pre[9])

    raw = scope.query(":WAV:DATA?")

    if raw.startswith("#"):
        nd = int(raw[1])
        nb = int(raw[2:2+nd])
        raw = raw[2+nd:2+nd+nb]

    adc = np.array([float(v) for v in raw.split(",") if v.strip() != ""])
    i = np.arange(adc.size)
    t = (i - xref) * xinc + xorig
    v = (adc - yref) * yinc + yorig
    return t, v

t1, v1 = read_waveform("CHAN1")
t2, v2 = read_waveform("CHAN2")

scope.write(":RUN")
scope.close()
rm.close()

def vpp(x): return float(np.max(x) - np.min(x))

Vpp_in  = vpp(v1)
Vpp_out = vpp(v2)
gain = Vpp_out / Vpp_in if Vpp_in else np.nan

# =========================
# Slew rate "rapport": ΔV / Δt sur la pente max
# =========================
dv = np.diff(v2)
dt = np.diff(t2)

slope = dv / dt                # V/s (signé)
abs_slope = np.abs(slope)

# Option anti-bruit : on prend le 99e percentile au lieu du max brut
target = np.quantile(abs_slope, 0.99)

# index du point le plus proche de ce target
k = int(np.argmin(np.abs(abs_slope - target)))

# fenêtre autour de k (en points)
W = 20  # ajuste 10-50 selon ton scope
i0 = max(0, k - W//2)
i1 = min(len(v2)-1, k + W//2)

tA, vA = t2[i0], v2[i0]
tB, vB = t2[i1], v2[i1]

dT = tB - tA
dV = vB - vA

SR = abs(dV / dT) / 1e6        # V/µs

print("\n=== MESURES ===")
print(f"Vpp_in  = {Vpp_in:.3f} V")
print(f"Vpp_out = {Vpp_out:.3f} V")
print(f"Gain    = {gain:.3f}")

print("\n=== SLEW RATE (méthode ΔV/Δt) ===")
print(f"tA = {tA*1e6:.3f} µs,  vA = {vA:.3f} V")
print(f"tB = {tB*1e6:.3f} µs,  vB = {vB:.3f} V")
print(f"Δt = {dT*1e6:.3f} µs")
print(f"ΔV = {dV:.3f} V")
print(f"Slew rate ≈ {SR:.3f} V/µs")

# =========================
# Plot + segment ΔV/Δt
# =========================
plt.figure()
plt.plot(t1*1e6, v1, label="CH1 (entrée)")
plt.plot(t2*1e6, v2, label="CH2 (sortie)")

# segment utilisé pour ΔV/Δt
plt.plot([tA*1e6, tB*1e6], [vA, vB], linewidth=3, label="Segment ΔV/Δt (SR)")

plt.xlabel("Temps (µs)")
plt.ylabel("Tension (V)")
plt.title("LM741 - Entrée vs Sortie (mesure Slew Rate)")
plt.grid(True)
plt.legend()
plt.show()


SCOPE: RIGOL TECHNOLOGIES,DS1104Z Plus,DS1ZC212301111,00.04.04.SP4


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.